In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
df1 = pd.read_excel("2019_bronx.xlsx")
df2 = pd.read_excel("2019_brooklyn.xlsx")
df3 = pd.read_excel("2019_manhattan.xlsx")
df4 = pd.read_excel("2019_queens.xlsx")
df5 = pd.read_excel("2019_statenisland.xlsx")

In [4]:
df1['AREA'] = 'bronx'
df2['AREA'] = 'brooklyn'
df3['AREA'] = 'manhattan'
df4['AREA'] = 'queens'
df5['AREA'] = 'statenisland'

In [5]:
frames = [df1, df2, df3, df4, df5]
df = pd.concat(frames)

In [6]:
df.columns = ['BOROUGH',
                  'NEIGHBORHOOD',
                  'BUILDING CLASS CATEGORY',
                  'TAX CLASS AS OF FINAL ROLL 18/19',
                  'BLOCK',
                  'LOT',
                  'EASE-MENT',
                  'BUILDING CLASS AS OF FINAL ROLL 18/19',
                  'ADDRESS',
                  'APARTMENT NUMBER',
                  'ZIP CODE',
                  'RESIDENTIAL UNITS',
                  'COMMERCIAL UNITS',
                  'TOTAL UNITS',
                  'LAND SQUARE FEET',
                  'GROSS SQUARE FEET',
                  'YEAR BUILT',
                  'TAX CLASS AT TIME OF SALE',
                  'BUILDING CLASS AT TIME OF SALE',
                  'SALE PRICE',
                  'SALE DATE',
                  'AREA']

In [7]:
df['FINAL_ADDRESS'] = df['ADDRESS'].apply(lambda x: x + " NYC")

In [8]:
df = df.drop(['EASE-MENT', 'APARTMENT NUMBER'], axis=1)

df = df[df['SALE PRICE'] < 3e6]
df = df[df['SALE PRICE'] > 5000]

df = df[df['RESIDENTIAL UNITS'] < 10.0]

df = df[df['COMMERCIAL UNITS'] < 5.0]

df = df[df['YEAR BUILT'] > 1860]
df = df[df['YEAR BUILT'] <= 2020]

df = df[df['GROSS SQUARE FEET'] < 4600]
df = df[df['GROSS SQUARE FEET'] > 50]

df = df[df['LAND SQUARE FEET'] > 200]
df = df[df['LAND SQUARE FEET'] < 7000]

llimit = np.percentile(df['ZIP CODE'], 1)
df = df[df['ZIP CODE'] > llimit]

df = df[df['TOTAL UNITS'] < 5]
df = df[df['TOTAL UNITS'] > 0]

df = df[df['LOT'] < 250]

In [9]:
df = df.dropna(how='any', axis=0)

In [10]:
df.shape[0]

22244

In [13]:
lat_long_exp = df.head(50)
lat_long_exp.shape[0]
lat_long_exp.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,AREA,FINAL_ADDRESS
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,70,A1,4445 PARK AVENUE,10457.0,1.0,0.0,1.0,1694.0,1497.0,1899.0,1,A1,388500,2019-07-23,bronx,4445 PARK AVENUE NYC
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,29,A9,2327 WASHINGTON AVENUE,10458.0,1.0,0.0,1.0,1095.0,1260.0,1910.0,1,A9,110000,2019-12-06,bronx,2327 WASHINGTON AVENUE NYC
2,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3043,55,A1,1948 BATHGATE AVENUE,10457.0,1.0,0.0,1.0,2356.0,2047.0,1901.0,1,A1,470000,2019-03-08,bronx,1948 BATHGATE AVENUE NYC


In [14]:
single_line_address = "4445 PARK AVENUE NYC"